In [2]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [6]:
import tweepy
import pandas as pd
import matplotlib.pyplot as plt
import re
import spacy

In [7]:
# Due to making this repository public, I have removed the keys after running the block

# Establish connection
auth = tweepy.OAuthHandler(secrets.API_Key, secrets.API_Secret_Key)
auth.set_access_token(secrets.Access_Token, secrets.Access_Token_Secret)

In [8]:
api = tweepy.API(auth)

In [52]:
# Cursor object allows us to navigate through Twitter
# This cursor object is setup to search for keywords 'tropical storm' and return 5 items

geoc='32.7765,79.9311,lmi'
storm_q_cursor = tweepy.Cursor(api.search, q='tropical storm', tweet_mode='extended', geoccode=geoc).items(5)

In [53]:
# print full text of all 5 tweets

[print(f'{count+1}: {i.full_text}') for count, i in enumerate(storm_q_cursor)]

1: RT @nytimes: The season’s fourth tropical storm, named Danny, made landfall on Monday evening over South Carolina, and then weakened into a…
2: Another interesting system: sub tropical storm #Raoni. Notice how it spins clockwise since it is south of the equator.🤯 Physics is fun https://t.co/OZ0bcqhsRq
3: Walked outside and got hit by lightning. Not directly but close enough to make me run inside and feel some heat, that was wild!!! Damn tropical storm Daniel. You asshole.
4: Tropical Storm Enrique, which spent the weekend as the first hurricane of the eastern Pacific season, kept losing steam Tuesday as it headed for the resorts at the southern end of Mexico's Baja California Peninsula.
https://t.co/XltjOApUdo
5: RT @weatherdak: The Brazilian Navy, which is the official agency for storms in this area, has dubbed this a subtropical storm.

A scientist…


[None, None, None, None, None]

In [94]:
# Get full information on multiple tweets containing a keyword within the geocode lon/lat coordinates
# This query searches 

geoc='32.7765,-79.9311,50mi'

number_of_tweets = 10
tweets = []
likes = []
time = []

for tweet in tweepy.Cursor(api.search, q='storm', tweet_mode='extended', geocode=geoc).items(number_of_tweets):
    tweets.append(tweet.full_text)
    likes.append(tweet.favorite_count)
    time.append(tweet.created_at)

df = pd.DataFrame({'tweets':tweets,'likes':likes,'time':time})
df

,tweets,likes,time
0,@lmstroud89 This could be the perfect storm. h...,3,2021-06-30 00:40:19
1,@MadamMoneySays Of course the infamous Lil Kim...,3,2021-06-29 23:34:56
2,RT @chswx: Isolated to scattered showers and s...,0,2021-06-29 22:33:52
3,Isolated to scattered showers and storms will ...,2,2021-06-29 22:30:56
4,"The 2nd school in Mississippi, the smallest SE...",0,2021-06-29 21:56:06
5,"@JustinKing The storm is coming, and we’ll be ...",1,2021-06-29 21:04:14
6,"RT @VincentGGraham: Earl’s Court, tandem parki...",0,2021-06-29 19:59:51
7,"Earl’s Court, tandem parking and storm water r...",3,2021-06-29 19:58:26
8,RT @carroxbil: #TheView Meghan McCain could re...,0,2021-06-29 19:00:02
9,#TheView Meghan McCain could realize the prote...,9,2021-06-29 18:36:23


In [106]:
# Remove retweets from query results
df_no_rt = df[~df.tweets.str.contains('RT')]
df_no_rt

,tweets,likes,time
0,@lmstroud89 This could be the perfect storm. h...,3,2021-06-30 00:40:19
1,@MadamMoneySays Of course the infamous Lil Kim...,3,2021-06-29 23:34:56
3,Isolated to scattered showers and storms will ...,2,2021-06-29 22:30:56
4,"The 2nd school in Mississippi, the smallest SE...",0,2021-06-29 21:56:06
5,"@JustinKing The storm is coming, and we’ll be ...",1,2021-06-29 21:04:14
7,"Earl’s Court, tandem parking and storm water r...",3,2021-06-29 19:58:26
9,#TheView Meghan McCain could realize the prote...,9,2021-06-29 18:36:23


In [104]:
# Optional parameter `geocode` will serve well in this project
# `geocode` parameter returns tweets by users located within a given radius of the given latitude/longitude
# This query searches keyword storm in South Carolina during tropical storm

tweet_lst = []
geoc='32.7765,-79.9311,50mi'

for tweet in tweepy.Cursor(api.search,q='storm',tweet_mode='extended',geocode=geoc).items(10):
    tweetDate = tweet.created_at.date()
    tweet_lst.append([tweetDate,
                      tweet.id,
                      tweet.user.screen_name,
                      tweet.user.name,
                      tweet.full_text])
tweet_df = pd.DataFrame(tweet_lst, columns=['tweet_dt', 'id', 'username', 'name', 'tweet'])

In [105]:
tweet_df

,tweet_dt,id,username,name,tweet
0,2021-06-30,1410035436022607874,alli_vapes,Allison,@lmstroud89 This could be the perfect storm. h...
1,2021-06-29,1410018982581747720,BigHassdaBossSC,Big Hass from da 843!!!,@MadamMoneySays Of course the infamous Lil Kim...
2,2021-06-29,1410003611405455361,QuintinOnCamera,Quintin Washington,RT @chswx: Isolated to scattered showers and s...
3,2021-06-29,1410002874462093317,chswx,Charleston Weather,Isolated to scattered showers and storms will ...
4,2021-06-29,1409994109977346052,Sctvman,Joe Wright,"The 2nd school in Mississippi, the smallest SE..."
5,2021-06-29,1409981057043161091,meredithchas,Meredith Blackwell,"@JustinKing The storm is coming, and we’ll be ..."
6,2021-06-29,1409964854060978188,edmcgrath,Edward McGrath,"RT @VincentGGraham: Earl’s Court, tandem parki..."
7,2021-06-29,1409964498425888768,VincentGGraham,Vince Graham,"Earl’s Court, tandem parking and storm water r..."
8,2021-06-29,1409949798325751814,ArleneSantana8,lenalena,RT @carroxbil: #TheView Meghan McCain could re...
9,2021-06-29,1409943846293807107,carroxbil,Carla Rox 🐞,#TheView Meghan McCain could realize the prote...
